Predicting Damage with Decision Trees

In [ ]:
import sqlite3
import warnings

import matplotlib.pyplot as plt
import pandas as pd
from category_encoders import OrdinalEncoder
from IPython.display import VimeoVideo
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.utils.validation import check_is_fitted

warnings.simplefilter(action="ignore", category=FutureWarning)

Prepare Data

In [ ]:
def wrangle(db_path):
    # Connect to database
    conn = sqlite3.connect(db_path)

    # Construct query
    query = """
        SELECT distinct(i.building_id) AS b_id,
           s.*,
           d.damage_grade
        FROM id_map AS i
        JOIN building_structure AS s ON i.building_id = s.building_id
        JOIN building_damage AS d ON i.building_id = d.building_id
        WHERE district_id = 4
    """

    # Read query results into DataFrame
    df = pd.read_sql(query, conn, index_col="b_id")

    # Identify leaky columns
    drop_cols = [col for col in df.columns if "post_eq" in col]

    # Add high-cardinality / redundant column
    drop_cols.append("building_id")

    # Create binary target column
    df["damage_grade"] = df["damage_grade"].str[-1].astype(int)
    df["severe_damage"] = (df["damage_grade"] > 3).astype(int)

    # Drop old target
    drop_cols.append("damage_grade")

    # Drop multicollinearity column
    drop_cols.append("count_floors_pre_eq")

    # Drop columns
    df.drop(columns=drop_cols, inplace=True)

    return df

In [ ]:
#Use the wrangle function above to import your data set into the DataFrame df. The path to the SQLite database is "/home/jovyan/nepal.sqlite"
df = wrangle("/home/jovyan/nepal.sqlite")
df.head()

Split

In [ ]:
#Create your feature matrix X and target vector y. Your target is "severe_damage"
target = "severe_damage"
X = df.drop(columns=target)
y = df[target]

In [ ]:
#Divide your data (X and y) into training and test sets using a randomized train-test split. Your test set should be 20% of your total data. And don't forget to set a random_state for reproducibility
X_train, X_test, y_train, y_test = train_test_split(
    X,y, test_size=0.2, random_state=42
)

In [ ]:
#Divide your training data (X_train and y_train) into training and validation sets using a randomized train-test split. Your validation data should be 20% of the remaining data. Don't forget to set a random_state
X_train, X_val, y_train, y_val = train_test_split(
    X_train,y_train, test_size=0.2, random_state=42
)

In [ ]:
#Calculate the baseline accuracy score for your model
acc_baseline = y_train.value_counts(normalize=True).max()
print("Baseline Accuracy:", round(acc_baseline, 2))

Iterate

In [ ]:
# Create a pipeline named model that contains a OrdinalEncoder transformer and a DecisionTreeClassifier predictor. (Be sure to set a random_state for your predictor.) Then fit your model to the training data.
# Build Model
model = make_pipeline(
    OrdinalEncoder(), DecisionTreeClassifier(random_state=42)
)
# Fit model to training data
model.fit(X_train, y_train)

In [ ]:
# Calculate the training and validation accuracy scores for your models
acc_train = accuracy_score(y_train, model.predict(X_train))
acc_val = model.score(X_val, y_val)

print("Training Accuracy:", round(acc_train, 2))
print("Validation Accuracy:", round(acc_val, 2))

In [ ]:
#Use the get_depth method on the DecisionTreeClassifier in your model to see how deep your tree grew during training.
tree_depth = model.named_steps["decisiontreeclassifier"].get_depth()
print("Tree Depth:", tree_depth)

In [ ]:
#Create a range of possible values for max_depth hyperparameter of your model's DecisionTreeClassifier. depth_hyperparams should range from 1 to 50 by steps of 2
depth_hyperparams = range(1,50,2)

In [ ]:
#Complete the code below so that it trains a model for every max_depth in depth_hyperparams. Every time a new model is trained, the code should also calculate the training and validation accuracy scores and append them to the training_acc and validation_acc lists, respectively.
# Create empty lists for training and validation accuracy scores
training_acc = []
validation_acc = []

for d in depth_hyperparams:
    # Create model with `max_depth` of `d`
    test_model = make_pipeline(
        OrdinalEncoder(), 
        DecisionTreeClassifier(max_depth=d,random_state=42)
    )
    # Fit model to training data
    test_model.fit(X_train, y_train)
    # Calculate training accuracy score and append to `training_acc`
    training_acc.append(test_model.score(X_train, y_train))
    # Calculate validation accuracy score and append to `training_acc`
    validation_acc.append(test_model.score(X_val, y_val))

print("Training Accuracy Scores:", training_acc[:3])
print("Validation Accuracy Scores:", validation_acc[:3])

In [ ]:
#Create a visualization with two lines. The first line should plot the training_acc values as a function of depth_hyperparams, and the second should plot validation_acc as a function of depth_hyperparams. You x-axis should be labeled "Max Depth", and the y-axis "Accuracy Score". Also include a legend so that your audience can distinguish between the two lines
# Plot `depth_hyperparams`, `training_acc`
plt.plot(depth_hyperparams, training_acc, label="training")
plt.plot(depth_hyperparams, validation_acc, label="validation")
plt.xlabel("Max depth")
plt.ylabel("accuracy score")
plt.legend()

In [ ]:
#Based on your visualization, choose the max_depth value that leads to the best validation accuracy score. Then retrain your original model with that max_depth value. Lastly, check how your tuned model performs on your test set by calculating the test accuracy score below. Were you able to resolve the overfitting problem with this new max_depth
model = make_pipeline(
    OrdinalEncoder(), DecisionTreeClassifier(max_depth=6,random_state=42)
)
# Fit model to training data
model.fit(X_train, y_train)

In [ ]:
# Calculate the training and validation accuracy scores for your models
acc_train = accuracy_score(y_train, model.predict(X_train))
acc_val = model.score(X_val, y_val)

print("Training Accuracy:", round(acc_train, 2))
print("Validation Accuracy:", round(acc_val, 2))

In [ ]:
test_acc = model.score(X_test, y_test)
print("Test Accuracy:", round(test_acc, 2))

Communicate

In [ ]:
#Complete the code below to use the plot_tree function from scikit-learn to visualize the decision logic of your model
# Create larger figure
fig, ax = plt.subplots(figsize=(25, 12))
# Plot tree
plot_tree(
    decision_tree=model.named_steps["decisiontreeclassifier"],
    feature_names= X_train.columns,
    filled=True,  # Color leaf with class
    rounded=True,  # Round leaf edges
    proportion=True,  # Display proportion of classes in leaf
    max_depth=3,  # Only display first 3 levels
    fontsize=12,  # Enlarge font
    ax=ax,  # Place in figure axis
);

In [ ]:
#Assign the feature names and importances of your model to the variables below. For the features, you can get them from the column names in your training set. For the importances, you access the feature_importances_ attribute of your model's DecisionTreeClassifier
features = X_train.columns
importances = model.named_steps["decisiontreeclassifier"].feature_importances_

print("Features:", features[:3])
print("Importances:", importances[:3])

In [ ]:
#Create a pandas Series named feat_imp, where the index is features and the values are your importances. The Series should be sorted from smallest to largest importance.
feat_imp = pd.Series(importances, index=features)
feat_imp.head()

In [ ]:
#Create a horizontal bar chart with all the features in feat_imp. Be sure to label your x-axis "Gini Importance"
# Create horizontal bar chart
feat_imp.plot(kind="barh")
plt.xlabel("Gini importance")
plt.ylabel("feature")